## How well do jobs and skills pay for Data Analysts?

#### Methology
    1.Evaluae median salary for top 6 data jobs
    2.Find median salary per skill for Data Analysts
    3.Visualize for highest paying skills and most demanded skills

In [44]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import seaborn as sns
import ast

#Loading dataset
ds = load_dataset('lukebarousse/data_jobs')
df = ds['train'].to_pandas()

# Changing job_posted_date to date time format
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [45]:
df_us = df[(df['job_country'] == 'United States')].dropna(subset = ['salary_year_avg'])

In [46]:
job_titles = df_us['job_title_short'].value_counts().index[:6].tolist()
job_titles

['Data Scientist',
 'Data Analyst',
 'Data Engineer',
 'Senior Data Scientist',
 'Senior Data Engineer',
 'Senior Data Analyst']

In [47]:
df_us_top6 = df_us[df_us['job_title_short'].isin(job_titles)]

In [48]:
job_order = df_us_top6.groupby('job_title_short')['salary_year_avg'].median().sort_values(ascending= False).index

In [49]:
sns.boxplot(data = df_us_top6, x = 'salary_year_avg', y = 'job_title_short', order = job_order)

ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, pos: f'${int(x/1000)}K'))
plt.xlim(0,600000)
plt.title('Salary Distribution in the United States')
plt.xlabel('Yearly Salary ($USD)')
plt.ylabel('')
plt.show()

### Investigate Median Salary Vs Skill for Data Analysts

In [50]:
df_Da_Us = df[(df['job_title_short'] == 'Data Analyst') & (df['job_country'] == 'United States')].copy()
df_Da_Us = df_Da_Us.dropna(subset= ['salary_year_avg'])

In [66]:
df_Da_Us = df_Da_Us.explode('job_skills')
df_Da_Us[['salary_year_avg', 'job_skills']].head(5)

salary_year_avg job_skills
109          89000.0     python
109          89000.0          r
109          89000.0    alteryx
109          89000.0    tableau
180          90250.0      excel

In [73]:
df_Da_Top_pay = df_Da_Us.groupby('job_skills')['salary_year_avg'].agg(['count','median']).sort_values(by = 'median', ascending= False)
df_Da_Top_pay = df_Da_Top_pay.head(10)
df_Da_Top_pay

count    median
job_skills                   
dplyr             2  196250.0
bitbucket         3  189000.0
gitlab            3  186000.0
solidity          1  179000.0
hugging face      1  175000.0
couchbase         1  160515.0
ansible           1  159640.0
mxnet             2  149000.0
cassandra         6  148250.0
vmware            1  147500.0

In [74]:
df_Da_skills = df_Da_Us.groupby('job_skills')['salary_year_avg'].agg(['count','median']).sort_values(by = 'count', ascending= False)

df_Da_skills = df_Da_skills.head(10).sort_values(by = 'median', ascending= False)

df_Da_skills

count    median
job_skills                 
python       1431  97500.00
tableau      1364  92875.00
r             893  92500.00
sql server    286  92500.00
sql          2508  91000.00
sas           926  90000.00
power bi      838  90000.00
powerpoint    462  85000.00
excel        1808  84392.00
word          461  81194.75

In [75]:
# # Create a figure with two vertically stacked subplots
fig, ax = plt.subplots(2,1)

# Set the Seaborn theme for better aesthetics
sns.set_theme(style='ticks')

# ------------------- First Bar Plot ------------------- #

# Create a horizontal bar plot for the highest-paid skills for Data Analysts
sns.barplot(data= df_Da_Top_pay, x='median', y = df_Da_Top_pay.index, ax = ax[0], hue = 'median', palette= 'dark:b_r', legend= False)

# Alternative approach (commented out) using Pandas built-in plotting function
# df_Da_Top_pay.plot(kind = 'barh', y = 'median', ax = ax[0], legend= False)

# Set the title for the first subplot
ax[0].set_title('Top 10  Highest Paid Skills for Data Analyst')

# Remove axis labels for a cleaner look
ax[0].set_xlabel('')
ax[0].set_ylabel('')

# Format x-axis labels to display salaries in thousands (e.g., "$50K" instead of "50000")
ax[0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x , pos: f'${int(x/1000)}K'))

# ------------------- Second Bar Plot ------------------- #

# Create a horizontal bar plot for the highest-paid in-demand skills for Data Analysts
sns.barplot(data= df_Da_skills, x='median', y = df_Da_skills.index, ax = ax[1], hue= 'median', palette= 'light:b', legend= False)

# Ensure both subplots share the same x-axis range for better comparison
ax[1].set_xlim(ax[0].get_xlim())

# Set the title for the second subplot
ax[1].set_title('Top 10 Highest Paid Demand Skills')

# Label the x-axis appropriately
ax[1].set_xlabel('Mediean Salary ($USD)')

# Remove the y-axis label for a cleaner look
ax[1].set_ylabel('')

# Format x-axis labels for salary in thousands (same as the first subplot)
ax[1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x , pos: f'${int(x/1000)}K'))


# ------------------- Final Formatting ------------------- #

# Adjust subplot spacing to prevent overlapping titles and labels
fig.tight_layout()
plt.show()
